In [232]:
import pandas as pd

In [233]:
df_mem=pd.read_csv('/content/memebers.csv')
df_sales=pd.read_csv('/content/sales.csv')
df_menu=pd.read_csv('/content/menu.csv')

In [234]:
df_sales

,customer_id,order_date,product_id
0,A,2021-01-01,1
1,A,2021-01-01,2
2,A,2021-01-07,2
3,A,2021-01-10,3
4,A,2021-01-11,3
5,A,2021-01-11,3
6,B,2021-01-01,2
7,B,2021-01-02,2
8,B,2021-01-04,1
9,B,2021-01-11,1


In [235]:
df_menu

,product_id,product_name,price
0,1,sushi,10
1,2,curry,15
2,3,ramen,12


In [236]:
df_mem

,customer_id,join_date
0,A,2021-01-07
1,B,2021-01-09


## What is the total amount each customer spent at the restaurant?

In [237]:
df_x=pd.merge(df_sales,df_menu,left_on="product_id",right_on="product_id",how="inner")


In [238]:
df_x.groupby(['customer_id'])['price'].sum()

customer_id
A    76
B    74
C    36
Name: price, dtype: int64

## 2. How many days has each customer visited the restaurant?

In [239]:
df_x.groupby(['customer_id'])['order_date'].nunique()

customer_id
A    4
B    6
C    2
Name: order_date, dtype: int64

### 3. What was the first item from the menu purchased by each customer?

In [240]:
t=df_x.groupby(['customer_id','order_date'])['product_name'].apply(sorted)

In [241]:
x=t.to_frame()
x=x.reset_index()

In [242]:
x['rank']=x['order_date'].rank(0,'dense')

In [243]:
x[x['rank']==1].explode(['product_name'])

,customer_id,order_date,product_name,rank
0,A,2021-01-01,curry,1.0
0,A,2021-01-01,sushi,1.0
4,B,2021-01-01,curry,1.0
10,C,2021-01-01,ramen,1.0
10,C,2021-01-01,ramen,1.0


4

In [244]:
res=df_x['product_name'].value_counts().reset_index()
res.rename(columns={"product_name":'count','index':'product_type'},inplace=True)
res.iloc[0]

product_type    ramen
count               8
Name: 0, dtype: object

5

In [245]:
t=df_x.groupby(['customer_id', 'product_name']).size().reset_index(name='count')

In [246]:
t['rank']=t.groupby(['customer_id'])['count'].rank("dense",ascending=False)

In [247]:
t[t['rank']==1][['customer_id','product_name','count']]

,customer_id,product_name,count
1,A,ramen,3
3,B,curry,2
4,B,ramen,2
5,B,sushi,2
6,C,ramen,3


In [248]:
t

,customer_id,product_name,count,rank
0,A,curry,2,2.0
1,A,ramen,3,1.0
2,A,sushi,1,3.0
3,B,curry,2,1.0
4,B,ramen,2,1.0
5,B,sushi,2,1.0
6,C,ramen,3,1.0


6. Which item was purchased first by the customer after they became a member?

In [249]:
df_merge=pd.merge(df_x,df_mem,on='customer_id',how="inner")

In [250]:
df_merge

,customer_id,order_date,product_id,product_name,price,join_date
0,A,2021-01-01,1,sushi,10,2021-01-07
1,A,2021-01-01,2,curry,15,2021-01-07
2,A,2021-01-07,2,curry,15,2021-01-07
3,A,2021-01-10,3,ramen,12,2021-01-07
4,A,2021-01-11,3,ramen,12,2021-01-07
5,A,2021-01-11,3,ramen,12,2021-01-07
6,B,2021-01-04,1,sushi,10,2021-01-09
7,B,2021-01-11,1,sushi,10,2021-01-09
8,B,2021-01-01,2,curry,15,2021-01-09
9,B,2021-01-02,2,curry,15,2021-01-09


In [251]:
df_merge['order_date']=pd.to_datetime(df_merge['order_date'])

In [252]:
df_merge['join_date']=pd.to_datetime(df_merge['join_date'])

In [253]:
df_merge=df_merge[df_merge['order_date']>df_merge['join_date']]

In [254]:
df_merge['rank']=df_merge.groupby(['customer_id'])['order_date'].rank('dense')

<ipython-input-254-e48e3509844d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['rank']=df_merge.groupby(['customer_id'])['order_date'].rank('dense')


In [255]:
df_merge

,customer_id,order_date,product_id,product_name,price,join_date,rank
3,A,2021-01-10,3,ramen,12,2021-01-07,1.0
4,A,2021-01-11,3,ramen,12,2021-01-07,2.0
5,A,2021-01-11,3,ramen,12,2021-01-07,2.0
7,B,2021-01-11,1,sushi,10,2021-01-09,1.0
10,B,2021-01-16,3,ramen,12,2021-01-09,2.0
11,B,2021-02-01,3,ramen,12,2021-01-09,3.0


In [256]:
df_merge[df_merge['rank']==1]

,customer_id,order_date,product_id,product_name,price,join_date,rank
3,A,2021-01-10,3,ramen,12,2021-01-07,1.0
7,B,2021-01-11,1,sushi,10,2021-01-09,1.0


7. Which item was purchased just before the customer became a member?

In [257]:
df_merge=pd.merge(df_x,df_mem,on='customer_id',how="inner")
df_merge['order_date']=pd.to_datetime(df_merge['order_date'])
df_merge['join_date']=pd.to_datetime(df_merge['join_date'])

In [258]:
df_merge

,customer_id,order_date,product_id,product_name,price,join_date
0,A,2021-01-01,1,sushi,10,2021-01-07
1,A,2021-01-01,2,curry,15,2021-01-07
2,A,2021-01-07,2,curry,15,2021-01-07
3,A,2021-01-10,3,ramen,12,2021-01-07
4,A,2021-01-11,3,ramen,12,2021-01-07
5,A,2021-01-11,3,ramen,12,2021-01-07
6,B,2021-01-04,1,sushi,10,2021-01-09
7,B,2021-01-11,1,sushi,10,2021-01-09
8,B,2021-01-01,2,curry,15,2021-01-09
9,B,2021-01-02,2,curry,15,2021-01-09


In [259]:
df_merge=df_merge[df_merge['order_date']<df_merge['join_date']]
df_merge['n_day']=(df_merge['join_date']-df_merge['order_date']).dt.days
df_merge

<ipython-input-259-f171092f7c9d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['n_day']=(df_merge['join_date']-df_merge['order_date']).dt.days


,customer_id,order_date,product_id,product_name,price,join_date,n_day
0,A,2021-01-01,1,sushi,10,2021-01-07,6
1,A,2021-01-01,2,curry,15,2021-01-07,6
6,B,2021-01-04,1,sushi,10,2021-01-09,5
8,B,2021-01-01,2,curry,15,2021-01-09,8
9,B,2021-01-02,2,curry,15,2021-01-09,7


In [260]:
df_merge['rank']=df_merge.groupby(['customer_id'])['n_day'].rank("dense")
df_merge[df_merge['rank']==1][['customer_id','order_date','product_name','price']]

<ipython-input-260-860505658de2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['rank']=df_merge.groupby(['customer_id'])['n_day'].rank("dense")


,customer_id,order_date,product_name,price
0,A,2021-01-01,sushi,10
1,A,2021-01-01,curry,15
6,B,2021-01-04,sushi,10


8.What is the total items and amount spent for each member before they became a member?

In [261]:
df_full=pd.merge(df_x,df_mem,on='customer_id',how="inner")

In [262]:
df_full['order_date']=pd.to_datetime(df_full['order_date'])
df_full['join_date']=pd.to_datetime(df_full['join_date'])

In [263]:
df_t=df_full[df_full['order_date']<df_full['join_date']]
df_t['total_prod']=1
df_t

<ipython-input-263-e10c9bd371cc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t['total_prod']=1


,customer_id,order_date,product_id,product_name,price,join_date,total_prod
0,A,2021-01-01,1,sushi,10,2021-01-07,1
1,A,2021-01-01,2,curry,15,2021-01-07,1
6,B,2021-01-04,1,sushi,10,2021-01-09,1
8,B,2021-01-01,2,curry,15,2021-01-09,1
9,B,2021-01-02,2,curry,15,2021-01-09,1


In [264]:
df_t.groupby(['customer_id'])['price','total_prod'].sum()

<ipython-input-264-5518f238bbbb>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_t.groupby(['customer_id'])['price','total_prod'].sum()


,price,total_prod
customer_id,,
A,25,2
B,40,3


If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

In [265]:
df_points=df_x.copy()

In [266]:
import numpy as np
df_points['points']=np.where(df_points['product_name']=='sushi',20*df_points['price'],df_points['price']*10)

In [267]:
df_points.groupby(['customer_id'])['points'].sum()

customer_id
A    860
B    940
C    360
Name: points, dtype: int64

10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [268]:
df_full=pd.merge(df_x,df_mem,on='customer_id',how="inner")
df_full['order_date']=pd.to_datetime(df_full['order_date'])
df_full['join_date']=pd.to_datetime(df_full['join_date'])

In [269]:
df_full['is_first_week']=df_full['join_date']+pd.Timedelta(days=7)

In [270]:
df_full['points_base']=df_full['price']*10

In [271]:
df_full[((df_full['order_date']>=df_full['join_date']) & (df_full['order_date']<df_full['is_first_week']))]

,customer_id,order_date,product_id,product_name,price,join_date,is_first_week,points_base
2,A,2021-01-07,2,curry,15,2021-01-07,2021-01-14,150
3,A,2021-01-10,3,ramen,12,2021-01-07,2021-01-14,120
4,A,2021-01-11,3,ramen,12,2021-01-07,2021-01-14,120
5,A,2021-01-11,3,ramen,12,2021-01-07,2021-01-14,120
7,B,2021-01-11,1,sushi,10,2021-01-09,2021-01-16,100


In [272]:
df_full['first_week_points']=np.where((df_full['order_date']>=df_full['join_date']) & (df_full['order_date']<df_full['is_first_week']),2*df_full['points_base'],df_full['points_base'])
df_full

,customer_id,order_date,product_id,product_name,price,join_date,is_first_week,points_base,first_week_points
0,A,2021-01-01,1,sushi,10,2021-01-07,2021-01-14,100,100
1,A,2021-01-01,2,curry,15,2021-01-07,2021-01-14,150,150
2,A,2021-01-07,2,curry,15,2021-01-07,2021-01-14,150,300
3,A,2021-01-10,3,ramen,12,2021-01-07,2021-01-14,120,240
4,A,2021-01-11,3,ramen,12,2021-01-07,2021-01-14,120,240
5,A,2021-01-11,3,ramen,12,2021-01-07,2021-01-14,120,240
6,B,2021-01-04,1,sushi,10,2021-01-09,2021-01-16,100,100
7,B,2021-01-11,1,sushi,10,2021-01-09,2021-01-16,100,200
8,B,2021-01-01,2,curry,15,2021-01-09,2021-01-16,150,150
9,B,2021-01-02,2,curry,15,2021-01-09,2021-01-16,150,150


get addtional 2x points on sushi

In [273]:
#df_full['t_points']=np.where((df_full['order_date']>df_full['join_date']) & (df_full['product_name']=='sushi'),2*df_full['points'],df_full['points'])

In [274]:
df_full['total_points']=np.where(df_full['product_name']=='sushi',2*df_full['points_base'],df_full['first_week_points'])

In [275]:
df_full

,customer_id,order_date,product_id,product_name,price,join_date,is_first_week,points_base,first_week_points,total_points
0,A,2021-01-01,1,sushi,10,2021-01-07,2021-01-14,100,100,200
1,A,2021-01-01,2,curry,15,2021-01-07,2021-01-14,150,150,150
2,A,2021-01-07,2,curry,15,2021-01-07,2021-01-14,150,300,300
3,A,2021-01-10,3,ramen,12,2021-01-07,2021-01-14,120,240,240
4,A,2021-01-11,3,ramen,12,2021-01-07,2021-01-14,120,240,240
5,A,2021-01-11,3,ramen,12,2021-01-07,2021-01-14,120,240,240
6,B,2021-01-04,1,sushi,10,2021-01-09,2021-01-16,100,100,200
7,B,2021-01-11,1,sushi,10,2021-01-09,2021-01-16,100,200,200
8,B,2021-01-01,2,curry,15,2021-01-09,2021-01-16,150,150,150
9,B,2021-01-02,2,curry,15,2021-01-09,2021-01-16,150,150,150


In [276]:
df_full=df_full[df_full['order_date'].dt.month==1]
df_full

,customer_id,order_date,product_id,product_name,price,join_date,is_first_week,points_base,first_week_points,total_points
0,A,2021-01-01,1,sushi,10,2021-01-07,2021-01-14,100,100,200
1,A,2021-01-01,2,curry,15,2021-01-07,2021-01-14,150,150,150
2,A,2021-01-07,2,curry,15,2021-01-07,2021-01-14,150,300,300
3,A,2021-01-10,3,ramen,12,2021-01-07,2021-01-14,120,240,240
4,A,2021-01-11,3,ramen,12,2021-01-07,2021-01-14,120,240,240
5,A,2021-01-11,3,ramen,12,2021-01-07,2021-01-14,120,240,240
6,B,2021-01-04,1,sushi,10,2021-01-09,2021-01-16,100,100,200
7,B,2021-01-11,1,sushi,10,2021-01-09,2021-01-16,100,200,200
8,B,2021-01-01,2,curry,15,2021-01-09,2021-01-16,150,150,150
9,B,2021-01-02,2,curry,15,2021-01-09,2021-01-16,150,150,150


In [277]:
df_full.groupby(['customer_id'])['total_points'].sum()

customer_id
A    1370
B     820
Name: total_points, dtype: int64

## Bonus Questions

In [278]:
df_full=df_full=pd.merge(df_x,df_mem,on='customer_id',how="outer")

In [279]:
df_full['join_date']=pd.to_datetime(df_full['join_date'])
df_full['order_date']=pd.to_datetime(df_full['order_date'])

In [280]:
df_full

,customer_id,order_date,product_id,product_name,price,join_date
0,A,2021-01-01,1,sushi,10,2021-01-07
1,A,2021-01-01,2,curry,15,2021-01-07
2,A,2021-01-07,2,curry,15,2021-01-07
3,A,2021-01-10,3,ramen,12,2021-01-07
4,A,2021-01-11,3,ramen,12,2021-01-07
5,A,2021-01-11,3,ramen,12,2021-01-07
6,B,2021-01-04,1,sushi,10,2021-01-09
7,B,2021-01-11,1,sushi,10,2021-01-09
8,B,2021-01-01,2,curry,15,2021-01-09
9,B,2021-01-02,2,curry,15,2021-01-09


In [281]:
df_full['member']=np.where(df_full['order_date']>=df_full['join_date'],'Y','N')

In [282]:
x=df_full.sort_values(['customer_id','order_date']).groupby(['customer_id','order_date','product_id','price','member']).head()

In [283]:
x

,customer_id,order_date,product_id,product_name,price,join_date,member
0,A,2021-01-01,1,sushi,10,2021-01-07,N
1,A,2021-01-01,2,curry,15,2021-01-07,N
2,A,2021-01-07,2,curry,15,2021-01-07,Y
3,A,2021-01-10,3,ramen,12,2021-01-07,Y
4,A,2021-01-11,3,ramen,12,2021-01-07,Y
5,A,2021-01-11,3,ramen,12,2021-01-07,Y
8,B,2021-01-01,2,curry,15,2021-01-09,N
9,B,2021-01-02,2,curry,15,2021-01-09,N
6,B,2021-01-04,1,sushi,10,2021-01-09,N
7,B,2021-01-11,1,sushi,10,2021-01-09,Y


In [284]:
x.columns

Index(['customer_id', 'order_date', 'product_id', 'product_name', 'price',
       'join_date', 'member'],
      dtype='object')

In [285]:
x[['customer_id', 'order_date', 'product_name', 'price', 'member']]

,customer_id,order_date,product_name,price,member
0,A,2021-01-01,sushi,10,N
1,A,2021-01-01,curry,15,N
2,A,2021-01-07,curry,15,Y
3,A,2021-01-10,ramen,12,Y
4,A,2021-01-11,ramen,12,Y
5,A,2021-01-11,ramen,12,Y
8,B,2021-01-01,curry,15,N
9,B,2021-01-02,curry,15,N
6,B,2021-01-04,sushi,10,N
7,B,2021-01-11,sushi,10,Y


## Bonus Question : 2

In [286]:
x_not_null=x[x['member']=='Y']
x_null=x[x['member']=='N']

In [287]:
x_null['ranking']="null"

<ipython-input-287-18dddabeebfa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_null['ranking']="null"


In [288]:
x_null

,customer_id,order_date,product_id,product_name,price,join_date,member,ranking
0,A,2021-01-01,1,sushi,10,2021-01-07,N,null
1,A,2021-01-01,2,curry,15,2021-01-07,N,null
8,B,2021-01-01,2,curry,15,2021-01-09,N,null
9,B,2021-01-02,2,curry,15,2021-01-09,N,null
6,B,2021-01-04,1,sushi,10,2021-01-09,N,null
12,C,2021-01-01,3,ramen,12,NaT,N,null
13,C,2021-01-01,3,ramen,12,NaT,N,null
14,C,2021-01-07,3,ramen,12,NaT,N,null


In [289]:
x_not_null

,customer_id,order_date,product_id,product_name,price,join_date,member
2,A,2021-01-07,2,curry,15,2021-01-07,Y
3,A,2021-01-10,3,ramen,12,2021-01-07,Y
4,A,2021-01-11,3,ramen,12,2021-01-07,Y
5,A,2021-01-11,3,ramen,12,2021-01-07,Y
7,B,2021-01-11,1,sushi,10,2021-01-09,Y
10,B,2021-01-16,3,ramen,12,2021-01-09,Y
11,B,2021-02-01,3,ramen,12,2021-01-09,Y


In [290]:
x_not_null.columns

Index(['customer_id', 'order_date', 'product_id', 'product_name', 'price',
       'join_date', 'member'],
      dtype='object')

In [291]:
x_not_null['ranking']=x_not_null.groupby(['customer_id'])['order_date'].rank("dense")

<ipython-input-291-b2e1529526fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_not_null['ranking']=x_not_null.groupby(['customer_id'])['order_date'].rank("dense")


In [292]:
x_not_null

,customer_id,order_date,product_id,product_name,price,join_date,member,ranking
2,A,2021-01-07,2,curry,15,2021-01-07,Y,1.0
3,A,2021-01-10,3,ramen,12,2021-01-07,Y,2.0
4,A,2021-01-11,3,ramen,12,2021-01-07,Y,3.0
5,A,2021-01-11,3,ramen,12,2021-01-07,Y,3.0
7,B,2021-01-11,1,sushi,10,2021-01-09,Y,1.0
10,B,2021-01-16,3,ramen,12,2021-01-09,Y,2.0
11,B,2021-02-01,3,ramen,12,2021-01-09,Y,3.0


In [293]:
df=pd.concat([x_not_null,x_null])
df

,customer_id,order_date,product_id,product_name,price,join_date,member,ranking
2,A,2021-01-07,2,curry,15,2021-01-07,Y,1.0
3,A,2021-01-10,3,ramen,12,2021-01-07,Y,2.0
4,A,2021-01-11,3,ramen,12,2021-01-07,Y,3.0
5,A,2021-01-11,3,ramen,12,2021-01-07,Y,3.0
7,B,2021-01-11,1,sushi,10,2021-01-09,Y,1.0
10,B,2021-01-16,3,ramen,12,2021-01-09,Y,2.0
11,B,2021-02-01,3,ramen,12,2021-01-09,Y,3.0
0,A,2021-01-01,1,sushi,10,2021-01-07,N,null
1,A,2021-01-01,2,curry,15,2021-01-07,N,null
8,B,2021-01-01,2,curry,15,2021-01-09,N,null


In [294]:
df_2=df.sort_values(['customer_id','order_date'],).groupby(['customer_id','order_date','price','member','ranking']).head()

In [295]:
df_2

,customer_id,order_date,product_id,product_name,price,join_date,member,ranking
0,A,2021-01-01,1,sushi,10,2021-01-07,N,null
1,A,2021-01-01,2,curry,15,2021-01-07,N,null
2,A,2021-01-07,2,curry,15,2021-01-07,Y,1.0
3,A,2021-01-10,3,ramen,12,2021-01-07,Y,2.0
4,A,2021-01-11,3,ramen,12,2021-01-07,Y,3.0
5,A,2021-01-11,3,ramen,12,2021-01-07,Y,3.0
8,B,2021-01-01,2,curry,15,2021-01-09,N,null
9,B,2021-01-02,2,curry,15,2021-01-09,N,null
6,B,2021-01-04,1,sushi,10,2021-01-09,N,null
7,B,2021-01-11,1,sushi,10,2021-01-09,Y,1.0


In [296]:
df_2[['customer_id', 'order_date', 'product_name', 'price', 'member','ranking']]

,customer_id,order_date,product_name,price,member,ranking
0,A,2021-01-01,sushi,10,N,null
1,A,2021-01-01,curry,15,N,null
2,A,2021-01-07,curry,15,Y,1.0
3,A,2021-01-10,ramen,12,Y,2.0
4,A,2021-01-11,ramen,12,Y,3.0
5,A,2021-01-11,ramen,12,Y,3.0
8,B,2021-01-01,curry,15,N,null
9,B,2021-01-02,curry,15,N,null
6,B,2021-01-04,sushi,10,N,null
7,B,2021-01-11,sushi,10,Y,1.0
